In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras import layers
import matplotlib.cm as cm
import random
import glob
import os
from skimage.segmentation import chan_vese
import numpy as np
import PIL 
from PIL import Image 
from tensorboard.plugins.hparams import api as hp
import pandas as pd
from shutil import copy
import cv2

# Display
# from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# My functions
import my_functions as mf

# tf explain
from tf_explain.callbacks.grad_cam import GradCAMCallback

In [6]:
import my_functions as mf

In [7]:
random_seed = 42

In [8]:
batch_size = 32
img_height = 180
img_width = 180
image_size = (img_height, img_width)

In [3]:
src_folder_path = "datasets\\ct_scan_3\\train\\COVID-positive\\"
des_folder_path = "datasets\\ct_scan_7\\train_contrast\\COVID-positive\\"
mf.aply_black_tophat_to_folder(src_folder_path, des_folder_path, disk_size=12)

Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_1.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_10.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_11.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_12.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_14.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_16.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_17.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_18.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_19.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_2.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_20.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_21.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_22.png
Current_image: datasets\ct_scan_3\train\COVID-positive\13_Bjorke_2

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
   'datasets/ct_scan_7/train_merge',
   labels='inferred',
   label_mode='int',
   class_names=None,
   color_mode='rgb',
   batch_size=batch_size,
   image_size=image_size,
   shuffle=True, 
   seed=random_seed, 
   interpolation='bilinear',
   follow_links=False,
   crop_to_aspect_ratio=False)

Found 23702 files belonging to 2 classes.


In [10]:
val_ds = tf.keras.utils.image_dataset_from_directory(
   'datasets/ct_scan_3/val',
   labels='inferred',
   label_mode='int',
   class_names=None,
   color_mode='rgb',
   batch_size=batch_size,
   image_size=image_size,
   shuffle=True, 
   seed=random_seed,
   interpolation='bilinear',
   follow_links=False,
   crop_to_aspect_ratio=False)

Found 2632 files belonging to 2 classes.


In [11]:
inputs = keras.Input(shape=(img_height, img_width, 3))

data_augmentation = keras.Sequential(
    [
      layers.RandomFlip("horizontal"), 
      layers.RandomRotation(0.1)
    ]
)
x = data_augmentation(inputs)  # Apply random data augmentation
# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1/127.5, offset=-1)
x = scale_layer(x)
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
base_model = keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(img_height, img_width, 3),
    input_tensor=x
)
last_layer_base_model = mf.find_last_layer(base_model)

x = tf.keras.layers.GlobalAveragePooling2D()(last_layer_base_model.output)

x = keras.layers.Dense(units=128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(units=64, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(units=32, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

base_model.trainable = False # Freeze base model

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 180, 180, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 180, 180, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 180, 180, 3)  0           sequential[0][0]                 
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 89, 89, 32)   864         rescaling[0][0]                  
______________________________________________________________________________________________

In [12]:
gcam_image_path = "datasets\\ct_scan_3\\val\\COVID-positive\\P021_106.png"

In [13]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(
    optimizer=optimizer,
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True)

run_logdir = mf.get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

data = (mf.get_img_array(gcam_image_path, image_size), None)
grd_cam_callback = GradCAMCallback(
   validation_data=data,
   class_index=0,
   output_dir=run_logdir,
)

epochs = 100
base_model.trainable = False
model.fit(
    train_ds, 
    epochs=epochs,
    batch_size=32,
    validation_data=val_ds,
    callbacks=[
      tensorboard_cb, 
      early_stopping_cb, 
      grd_cam_callback])

Epoch 1/100
741/741 [==============================] - 119s 143ms/step - loss: 0.6503 - accuracy: 0.6124 - val_loss: 0.4736 - val_accuracy: 0.8233
Epoch 2/100
741/741 [==============================] - 99s 133ms/step - loss: 0.5680 - accuracy: 0.6684 - val_loss: 0.4211 - val_accuracy: 0.8267
Epoch 3/100
741/741 [==============================] - 101s 136ms/step - loss: 0.5560 - accuracy: 0.6690 - val_loss: 0.3847 - val_accuracy: 0.8404
Epoch 4/100
741/741 [==============================] - 103s 139ms/step - loss: 0.5523 - accuracy: 0.6721 - val_loss: 0.4145 - val_accuracy: 0.8370
Epoch 5/100
741/741 [==============================] - 104s 140ms/step - loss: 0.5413 - accuracy: 0.6801 - val_loss: 0.3812 - val_accuracy: 0.8518
Epoch 6/100
241/741 [========>.....................] - ETA: 1:03 - loss: 0.5390 - accuracy: 0.6756

KeyboardInterrupt: 